### Dataset: Amazon Music
### Algorithm: Time-Aware ItemKNN Incremental Vaz 2013
### Padding unknown user/item: RecentPop
### Flow: Library Flow

In [1]:
k = 100
from streamsight.datasets.amazon import AmazonMusicDataset
from streamsight.preprocessing import MinItemsPerUser
from streamsight.settings import SlidingWindowSetting
dataset = AmazonMusicDataset()
dataset.add_filter(
    MinItemsPerUser(3, AmazonMusicDataset.ITEM_IX, AmazonMusicDataset.USER_IX)
)
data = dataset.load()

setting_window = SlidingWindowSetting(
    background_t=1546241746,
    window_size=60 * 60 * 24 * 365, # 365 days
    top_K=k
)

setting_window.split(data)

INFO - streamsight package loaded.
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

DEBUG - AmazonMusicDataset being initialized with 'data' as the base path.
DEBUG - AmazonMusicDataset is initialized.
INFO - AmazonMusicDataset is loading dataset...
DEBUG - Data file is in memory and in dir specified.
DEBUG - AmazonMusicDataset applying filters set.
DEBUG - 	interactions before preprocess: 130434
DEBUG - 	items before preprocess: 70511
DEBUG - 	users before preprocess: 100952
DEBUG - applying filter: MinItemsPerUser(min_iu=3, count_duplicates=True, item_ix=parent_asin, user_ix=user_id)
DEBUG - 	interactions after filter: 25542
DEBUG - 	items after filter: 20836
DEBUG - 	users after filter: 4464
DEBUG - 	interactions after preprocess: 25542
DEBUG - 

  0%|          | 0/4 [00:00<?, ?it/s]

DEBUG - NPastInteractionTimestampSplitter(t=1546241746,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) - Updating split point to t=1546241746
DEBUG - Performing lt(t, 1577777746)
DEBUG - Performing ge(t, 1546241746)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=1546241746,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 1546241746 resulted in empty unlabelled testing samples.
DEBUG - NPastInteractionTimestampSplitter(t=1546241746,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) - Updating split point to t=1577777746
DEBUG - Performing lt(t, 1609313746)
DEBUG - Performing ge(t, 1577777746)
DEBUG - Performing get_user_n_last_interaction comparison
DEBUG - NPastInteractionTimestampSplitter(t=1577777746,t_lower=None,t_upper=31536000,n_seq_data=0,include_all_past_data=False) has complete split
INFO - Split at time 1577777746 re

5it [00:00, 81.62it/s]               

INFO - Finished split with window size 31536000 seconds. Number of splits: 5 in total.
INFO - SlidingWindowSetting data split - Took 0.0884s
DEBUG - Checking split attribute and sizes.
DEBUG - Checking split attributes.
DEBUG - Split attributes are set.
DEBUG - Checking size of split sets.
DEBUG - Size of split sets are checked.
INFO - SlidingWindowSetting data split complete.


In [2]:
from streamsight.evaluators import EvaluatorStreamerBuilder

builder = EvaluatorStreamerBuilder()
builder.add_setting(setting_window)
builder.set_metric_K(k)
builder.add_metric("PrecisionK")
builder.add_metric("RecallK")
builder.add_metric("NDCGK")
evaluator = builder.build()

In [3]:
from streamsight.algorithms import TARSItemKNNVaz
external_model = TARSItemKNNVaz(K=k)
external_model_id = evaluator.register_algorithm(algorithm_name="AmazonMusic_TA_ItemKNN_Inc_Vaz")
print(external_model_id)

INFO - Registering algorithm name AmazonMusic_TA_ItemKNN_Inc_Vaz with ID: bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - Algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d registered
bdd640fb-0667-4ad1-9c80-317fa3b1799d


In [4]:
evaluator.start_stream()

DEBUG - Resetting data generators.
DEBUG - Data generators are reset.
DEBUG - Preparing evaluator for streaming
DEBUG - (user x item) shape defined is (3733, 16060)
DEBUG - Shape of dataframe stored in matrix was (19253, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (19253, 6) after masking
DEBUG - Final (user x item) shape defined is (3733, 16060)
DEBUG - Caching evaluation data for step 1
DEBUG - (user x item) shape defined is (3733, 16060)
DEBUG - Shape of dataframe stored in matrix was (1719, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1719, 6) after masking
DEBUG - Final (user x item) shape defined is (3733, 16060)
DEBUG - (user x item) shape defined is (3733, 16060)
DEBUG - Shape of dataframe stored in matrix was (1719, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (295, 6) after masking
DEBUG - Final (user x item) shape defined is (3957, 16060)
DEBUG - Data cached for step 1 complete


# Iteration 1

In [5]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
X after decay:  [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.35338923 0.         0.        ]
 [0.         0.         0.         ... 0.         0.36292292 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: TARSItemKNNVaz missing similar items for 15947 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Fitting TARSItemKNNVaz complete - Took 0.19s


       interactionid   uid    iid          ts  rating  helpful_vote
0                  0     0      0   906574660     5.0             1
1                  1     0      1   908892059     3.0             0
2                  2     1      2   920935758     5.0             0
3                  3     2      3   941546039     4.0             0
4                  4     1      4   942757962     5.0            20
...              ...   ...    ...         ...     ...           ...
19248          19248  3730  15828  1546039809     5.0             1
19249          19249  3730  16057  1546039948     5.0             0
19250          19250  3731  16058  1546042248     4.0             0
19251          19251  1620  16059  1546057175     5.0             2
19252          19252  3732  14415  1546129819     3.0             1

[19253 rows x 6 columns]

In [6]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: TARSItemKNNVaz missing similar items for 15947 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Max user ID:  3957
Max item ID:  0
X.shape:  (3733, 16060)
Intended shape:  (3957, 16060)
Predict frame:         interactionid   uid  iid          ts  rating  helpful_vote
19253          19253  3732   -1  1546243901     3.0             0
19254          19254  3733   -1  1546295208     5.0             0
19255          19255  1561   -1  1546307829     5.0             7
19256          19256  1787   -1  1546308024     1.0             0

       interactionid   uid  iid          ts  rating  helpful_vote
19253          19253  3732   -1  1546243901     3.0             0
19254          19254  3733   -1  1546295208     5.0             0
19255          19255  1561   -1  1546307829     5.0             7
19256          19256  1787   -1  1546308024     1.0             0
19257          19257  1787   -1  1546308362     1.0             0
...              ...   ...  ...         ...     ...           ...
20967          20967   212   -1  1577729682     5.0             0
20968          20968   522   -1  1577753078     5.0             0
20969          20969  3076   -1  1577754015     5.0             0
20970          20970   753   -1  1577754537     5.0             0
20971          20971  3288   -1  1577755529     5.0             0

[1719 rows x 6 columns]

In [7]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3597964 stored elements and shape (3957, 16060)>
  Coords	Values
  (221, 4207)	1.878993321857759e-138
  (221, 8238)	1.8789933147700823e-138
  (309, 6723)	3.9746664866374526e-199
  (395, 3650)	1.4668133738978e-310
  (441, 11769)	-1.9960384511088632e-107
  (441, 12492)	-4.990096127768771e-107
  (441, 13082)	-1.7286200055849535e-107
  (441, 14423)	-5.185860016766846e-107
  (441, 15575)	1.7286200055849535e-107
  (646, 5819)	-6.799435036416674e-99
  (646, 12492)	1.5391367982648862e-218
  (646, 15026)	-5.441670336140416e-219
  (646, 11769)	-3.925655648612539e-99
  (646, 13613)	-5.708889198541013e-187
  (814, 4207)	2.196242730039434e-270
  (814, 5681)	-2.6898370098006307e-270
  (814, 12457)	1.5529781216846123e-270
  (814, 14097)	-2.689837009800631e-270
  (814, 15325)	2.689837009800631e-270
  (1002, 15329)	4.142787086601579e-63
  (1033, 15610)	-7.371858067631027e-41
  (1033, 15754)	-6.6226005881418735e-68
  (1135, 14495)	-2.49491639

# Iteration 2

In [8]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (3957, 17393)
DEBUG - Shape of dataframe stored in matrix was (1719, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1719, 6) after masking
DEBUG - Final (user x item) shape defined is (3957, 17393)
DEBUG - Caching evaluation data for step 2
DEBUG - (user x item) shape defined is (3957, 17393)
DEBUG - Shape of dataframe stored in matrix was (1267, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1267, 6) after masking
DEBUG - Final (user x item) shape defined is (3957, 17393)
DEBUG - (user x item) shape defined is (3957, 17393)
DEBUG - Shape of dataframe stored in matrix was (1267, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (234, 6) after masking
DEBUG - Final (user x item) shape defined is (4109, 17393)
DEBUG - Data cached for step 2 complete
X after decay:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]


       interactionid   uid    iid          ts  rating  helpful_vote
19253          19253  3732  16060  1546243901     3.0             0
19254          19254  3733  16061  1546295208     5.0             0
19255          19255  1561  16062  1546307829     5.0             7
19256          19256  1787  16063  1546308024     1.0             0
19257          19257  1787  16064  1546308362     1.0             0
...              ...   ...    ...         ...     ...           ...
20967          20967   212   7561  1577729682     5.0             0
20968          20968   522  17389  1577753078     5.0             0
20969          20969  3076  17390  1577754015     5.0             0
20970          20970   753  17391  1577754537     5.0             0
20971          20971  3288  17392  1577755529     5.0             0

[1719 rows x 6 columns]

In [9]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: TARSItemKNNVaz missing similar items for 17247 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Max user ID:  4111
Max item ID:  0
X.shape:  (3957, 17393)
Intended shape:  (4111, 17393)
Predict frame:         interactionid   uid  iid          ts  rating  helpful_vote
20972          20972   753   -1  1577895301     5.0             5
20973          20973  3551   -1  1577900463     5.0             0
20974          20974  3551   -1  1577900694     4.0             0
20975          20975  3957   -1  1577912436     5.0             0

       interactionid   uid  iid          ts  rating  helpful_vote
20972          20972   753   -1  1577895301     5.0             5
20973          20973  3551   -1  1577900463     5.0             0
20974          20974  3551   -1  1577900694     4.0             0
20975          20975  3957   -1  1577912436     5.0             0
20976          20976  3957   -1  1577912562     5.0             2
...              ...   ...  ...         ...     ...           ...
22234          22234  4077   -1  1609103063     5.0             0
22235          22235  4058   -1  1609183165     5.0             1
22236          22236  1265   -1  1609209509     5.0             2
22237          22237  2296   -1  1609217665     5.0             0
22238          22238  4102   -1  1609263547     5.0             0

[1267 rows x 6 columns]

In [10]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2679226 stored elements and shape (4111, 17393)>
  Coords	Values
  (221, 12457)	1.148437421370655e-207
  (221, 15684)	8.573205005845989e-297
  (271, 8283)	1.5695642515115007e-156
  (302, 15928)	1.3918960373075598e-263
  (402, 15972)	1.4391692698016036e-208
  (441, 15654)	6.884427848968663e-266
  (512, 10713)	2.0225579601262505e-139
  (517, 5860)	7.6219e-319
  (544, 14622)	1.3432849667131205e-267
  (659, 8728)	1.9611755152187355e-236
  (707, 16549)	-5.621629770872201e-233
  (715, 15108)	-2.1098655773198278e-50
  (715, 15295)	9.30363269560664e-51
  (731, 6450)	9.461275307918463e-33
  (735, 15743)	1.1061398528989408e-16
  (735, 14707)	1.463285483014083e-16
  (735, 12666)	-2.835502647390986e-295
  (735, 14828)	1.101790630047625e-283
  (735, 15458)	-6.3508323883347455e-233
  (735, 15469)	-1.068e-320
  (735, 15745)	2.6749340788944908e-104
  (735, 16053)	7.2216432104555635e-106
  (772, 15684)	4.867355656497673e-52
  (812, 5860)	1.1

# Iteration 3

In [11]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (4111, 18338)
DEBUG - Shape of dataframe stored in matrix was (1267, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1267, 6) after masking
DEBUG - Final (user x item) shape defined is (4111, 18338)
DEBUG - Caching evaluation data for step 3
DEBUG - (user x item) shape defined is (4111, 18338)
DEBUG - Shape of dataframe stored in matrix was (1597, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1597, 6) after masking
DEBUG - Final (user x item) shape defined is (4111, 18338)
DEBUG - (user x item) shape defined is (4111, 18338)
DEBUG - Shape of dataframe stored in matrix was (1597, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (254, 6) after masking
DEBUG - Final (user x item) shape defined is (4309, 18338)
DEBUG - Data cached for step 3 complete
X after decay:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]


       interactionid   uid    iid          ts  rating  helpful_vote
20972          20972   753  17393  1577895301     5.0             5
20973          20973  3551  17394  1577900463     5.0             0
20974          20974  3551  17395  1577900694     4.0             0
20975          20975  3957  17396  1577912436     5.0             0
20976          20976  3957  17397  1577912562     5.0             2
...              ...   ...    ...         ...     ...           ...
22234          22234  4077  14828  1609103063     5.0             0
22235          22235  4058  18335  1609183165     5.0             1
22236          22236  1265  18336  1609209509     5.0             2
22237          22237  2296  18337  1609217665     5.0             0
22238          22238  4102  17660  1609263547     5.0             0

[1267 rows x 6 columns]

In [12]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: TARSItemKNNVaz missing similar items for 18225 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Max user ID:  4309
Max item ID:  0
X.shape:  (4111, 18338)
Intended shape:  (4309, 18338)
Predict frame:         interactionid   uid  iid          ts  rating  helpful_vote
22239          22239   791   -1  1609352713     2.0             0
22240          22240  2342   -1  1609352784     5.0             1
22241          22241   791   -1  1609352935     1.0             0
22242          22242  3633   -1  1609429831     5.0             0

       interactionid   uid  iid          ts  rating  helpful_vote
22239          22239   791   -1  1609352713     2.0             0
22240          22240  2342   -1  1609352784     5.0             1
22241          22241   791   -1  1609352935     1.0             0
22242          22242  3633   -1  1609429831     5.0             0
22243          22243  1127   -1  1609435015     4.0             0
...              ...   ...  ...         ...     ...           ...
23831          23831  4307   -1  1640701893     5.0             0
23832          23832  4307   -1  1640702119     5.0             0
23833          23833  4307   -1  1640703766     5.0             1
23834          23834  4308   -1  1640709415     5.0             2
23835          23835  3959   -1  1640710325     5.0             1

[1597 rows x 6 columns]

In [13]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3631775 stored elements and shape (4309, 18338)>
  Coords	Values
  (462, 17826)	1.2418688445750985e-53
  (711, 9414)	3.8144631240917195e-196
  (711, 16047)	2.335872074171835e-195
  (735, 13870)	-2.480149591221351e-181
  (735, 14707)	1.109156616076292e-181
  (735, 12666)	-4.4751749535844714e-263
  (735, 15554)	-6.328853113351545e-263
  (735, 15931)	1.0961895145965349e-262
  (735, 15962)	1.45012242270952e-262
  (735, 16053)	3.6539650486829223e-264
  (735, 17223)	1.2657706226703089e-262
  (735, 17238)	2.192379029209753e-263
  (735, 17683)	-6.328853113351544e-263
  (735, 17740)	-4.475174953584472e-263
  (735, 17845)	1.0961895145965349e-262
  (735, 17848)	-6.328848903505601e-263
  (735, 17961)	1.0961895145965346e-262
  (735, 18014)	-2.192379029126334e-263
  (767, 16364)	-1.8664414736724435e-198
  (767, 17312)	4.3823633370974123e-191
  (814, 15593)	1.1295418466126595e-283
  (859, 11803)	-1.4263640083746378e-299
  (859, 12666)	-4.0

# Iteration 4

In [14]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (4309, 19548)
DEBUG - Shape of dataframe stored in matrix was (1597, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1597, 6) after masking
DEBUG - Final (user x item) shape defined is (4309, 19548)
DEBUG - Caching evaluation data for step 4
DEBUG - (user x item) shape defined is (4309, 19548)
DEBUG - Shape of dataframe stored in matrix was (1259, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1259, 6) after masking
DEBUG - Final (user x item) shape defined is (4309, 19548)
DEBUG - (user x item) shape defined is (4309, 19548)
DEBUG - Shape of dataframe stored in matrix was (1259, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (197, 6) after masking
DEBUG - Final (user x item) shape defined is (4424, 19548)
DEBUG - Data cached for step 4 complete
X after decay:  [[0.         0.         0.         ... 0.         

       interactionid   uid    iid          ts  rating  helpful_vote
22239          22239   791  18338  1609352713     2.0             0
22240          22240  2342  18339  1609352784     5.0             1
22241          22241   791  18340  1609352935     1.0             0
22242          22242  3633  18341  1609429831     5.0             0
22243          22243  1127  18342  1609435015     4.0             0
...              ...   ...    ...         ...     ...           ...
23831          23831  4307  19547  1640701893     5.0             0
23832          23832  4307  11670  1640702119     5.0             0
23833          23833  4307  12786  1640703766     5.0             1
23834          23834  4308  12399  1640709415     5.0             2
23835          23835  3959  18308  1640710325     5.0             1

[1597 rows x 6 columns]

In [15]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: TARSItemKNNVaz missing similar items for 19418 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Max user ID:  4431
Max item ID:  0
X.shape:  (4309, 19548)
Intended shape:  (4431, 19548)
Predict frame:         interactionid   uid  iid          ts  rating  helpful_vote
23836          23836  4230   -1  1640869031     5.0             1
23837          23837  1548   -1  1640883197     5.0             1
23838          23838  1677   -1  1640884401     5.0             4
23839          23839  1677   -1  1640885026     5.0             0

       interactionid   uid  iid          ts  rating  helpful_vote
23836          23836  4230   -1  1640869031     5.0             1
23837          23837  1548   -1  1640883197     5.0             1
23838          23838  1677   -1  1640884401     5.0             4
23839          23839  1677   -1  1640885026     5.0             0
23840          23840  1680   -1  1640885425     3.0             1
...              ...   ...  ...         ...     ...           ...
25090          25090  2489   -1  1672339062     5.0             0
25091          25091  4129   -1  1672362059     5.0             0
25092          25092  4129   -1  1672362059     5.0             0
25093          25093  3250   -1  1672370323     5.0             0
25094          25094  4176   -1  1672374881     5.0             0

[1259 rows x 6 columns]

In [16]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2385875 stored elements and shape (4431, 19548)>
  Coords	Values
  (22, 17875)	1.0804126528181977e-302
  (500, 18364)	4.1125857153402195e-140
  (500, 18743)	-2.3744024698170854e-140
  (942, 18539)	-7.355289438306764e-117
  (942, 18634)	-1.040195007883301e-116
  (1011, 9)	-3.0640322406263175e-61
  (1011, 15777)	7.993656126094306e-46
  (1011, 17091)	-6.191859410334429e-45
  (1011, 18539)	1.9580378688203916e-45
  (1024, 19121)	3.0422546577096415e-121
  (1024, 0)	4.318978996510957e-133
  (1024, 9)	7.480691058779822e-133
  (1024, 1291)	-1.5211273288623014e-121
  (1024, 15777)	1.2467818431299702e-133
  (1024, 17091)	-9.657530629592327e-133
  (1024, 17687)	1.2348042898449473e-306
  (1024, 18539)	-1.246603731040732e-78
  (1024, 18634)	-1.7629639033427054e-78
  (1157, 16975)	3.953468063154136e-173
  (1391, 17743)	4.003757986993011e-124
  (1391, 17913)	9.281362228281054e-292
  (1391, 18959)	8.952675022600884e-124
  (1573, 18256)	1.007

# Iteration 5

In [17]:
data = evaluator.get_data(external_model_id)
external_model.fit(data)
data

DEBUG - Getting data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
DEBUG - (user x item) shape defined is (4431, 20527)
DEBUG - Shape of dataframe stored in matrix was (1259, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (1259, 6) after masking
DEBUG - Final (user x item) shape defined is (4431, 20527)
DEBUG - Caching evaluation data for step 5
DEBUG - (user x item) shape defined is (4431, 20527)
DEBUG - Shape of dataframe stored in matrix was (447, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (447, 6) after masking
DEBUG - Final (user x item) shape defined is (4431, 20527)
DEBUG - (user x item) shape defined is (4431, 20527)
DEBUG - Shape of dataframe stored in matrix was (447, 6) before masking
DEBUG - Shape of dataframe stored in matrix is now (105, 6) after masking
DEBUG - Final (user x item) shape defined is (4462, 20527)
DEBUG - Data cached for step 5 complete
X after decay:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0

       interactionid   uid    iid          ts  rating  helpful_vote
23836          23836  4230  18308  1640869031     5.0             1
23837          23837  1548  18862  1640883197     5.0             1
23838          23838  1677  19548  1640884401     5.0             4
23839          23839  1677  17482  1640885026     5.0             0
23840          23840  1680  19549  1640885425     3.0             1
...              ...   ...    ...         ...     ...           ...
25090          25090  2489  20523  1672339062     5.0             0
25091          25091  4129  20524  1672362059     5.0             0
25092          25092  4129  20524  1672362059     5.0             0
25093          25093  3250  20525  1672370323     5.0             0
25094          25094  4176  20526  1672374881     5.0             0

[1259 rows x 6 columns]

In [18]:
ul_data = evaluator.get_unlabeled_data(external_model_id)
prediction = external_model.predict(data, ul_data)
ul_data

DEBUG - Getting unlabeled data for algorithm bdd640fb-0667-4ad1-9c80-317fa3b1799d
WARNING - /Users/joshua/projects/fyp/streamsightv2/venv/lib/python3.12/site-packages/streamsight/algorithms/base.py:315: UserWarning: TARSItemKNNVaz missing similar items for 20401 items.
  warn(f"{self.name} missing similar items for {missing} items.")

DEBUG - Performing items_in comparison
In ItemKNNIncremental _predict:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Max user ID:  4464
Max item ID:  0
X.shape:  (4431, 20527)
Intended shape:  (4464, 20527)
Predict frame:         interactionid   uid  iid          ts  rating  helpful_vote
25095          25095  4339   -1  1672406677     5.0             0
25096          25096  4339   -1  1672406685     5.0             0
25097          25097  4279   -1  1672427352     5.0             0
25098          25098  4279   -1  1672427477     5.0             0

       interactionid   uid  iid          ts  rating  helpful_vote
25095          25095  4339   -1  1672406677     5.0             0
25096          25096  4339   -1  1672406685     5.0             0
25097          25097  4279   -1  1672427352     5.0             0
25098          25098  4279   -1  1672427477     5.0             0
25099          25099  1466   -1  1672440344     5.0             0
...              ...   ...  ...         ...     ...           ...
25537          25537  4462   -1  1691639002     5.0             0
25538          25538  4463   -1  1692200640     5.0             0
25539          25539  4463   -1  1692202028     5.0             0
25540          25540  4463   -1  1692202101     5.0             0
25541          25541  2896   -1  1692267495     5.0             0

[447 rows x 6 columns]

In [19]:
print(prediction)
print(prediction.toarray())
evaluator.submit_prediction(external_model_id, prediction)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 678065 stored elements and shape (4464, 20527)>
  Coords	Values
  (133, 18736)	-2.0810546415374197e-16
  (133, 18944)	-6.580872602523496e-17
  (133, 18275)	-4.0622807366274016e-249
  (133, 18573)	1.6479774205241777e-249
  (133, 18773)	1.6584191475418074e-249
  (133, 18891)	-1.6249122946509604e-248
  (133, 18955)	2.345358876815662e-249
  (133, 19297)	-2.6253237592396604e-265
  (133, 19591)	-4.0622807366274016e-249
  (133, 19638)	-4.0622807366274016e-249
  (133, 19686)	2.3453588768156613e-249
  (133, 19713)	-4.062280736627401e-249
  (133, 19852)	4.062280736627401e-249
  (133, 19988)	4.0622807366274016e-249
  (169, 19086)	1.2970043156189422e-153
  (170, 17902)	3.41963658371023e-39
  (170, 18482)	1.65579113858556e-201
  (170, 18871)	9.86426361999912e-237
  (268, 18736)	3.3743761645392365e-157
  (285, 19656)	-1.362030411701102e-294
  (342, 18721)	2.162303510298351e-58
  (342, 18736)	9.67011527402706e-59
  (342, 19118)	3.925689533

# evaluate metrics

In [20]:
evaluator.get_all_algorithm_status()

{'AmazonMusic_TA_ItemKNN_Inc_Vaz_bdd640fb-0667-4ad1-9c80-317fa3b1799d': <AlgorithmStateEnum.COMPLETED: 'COMPLETED'>}

In [21]:
evaluator.metric_results("macro")

macro_score  \
Algorithm                                          Metric                        
AmazonMusic_TA_ItemKNN_Inc_Vaz_bdd640fb-0667-4a... NDCGK_100          0.026580   
                                                   PrecisionK_100     0.001038   
                                                   RecallK_100        0.065244   

                                                                   num_window  
Algorithm                                          Metric                      
AmazonMusic_TA_ItemKNN_Inc_Vaz_bdd640fb-0667-4a... NDCGK_100                5  
                                                   PrecisionK_100           5  
                                                   RecallK_100              5

In [22]:
evaluator.metric_results("micro")


micro_score  \
Algorithm                                          Metric                       
AmazonMusic_TA_ItemKNN_Inc_Vaz_bdd640fb-0667-4a... NDCGK_100         0.025156   
                                                   PrecisionK_100    0.000986   
                                                   RecallK_100       0.062692   

                                                                   num_user  
Algorithm                                          Metric                    
AmazonMusic_TA_ItemKNN_Inc_Vaz_bdd640fb-0667-4a... NDCGK_100            761  
                                                   PrecisionK_100       761  
                                                   RecallK_100          761

In [23]:
evaluator.metric_results("user")

user_id  \
Algorithm                                          Timestamp    Metric                   
AmazonMusic_TA_ItemKNN_Inc_Vaz_bdd640fb-0667-4a... t=1546241746 PrecisionK_100     512   
                                                                PrecisionK_100    1024   
                                                                PrecisionK_100    3587   
                                                                PrecisionK_100    3590   
                                                                PrecisionK_100    3079   
...                                                                                ...   
                                                   t=1672385746 NDCGK_100         3956   
                                                                NDCGK_100         4348   
                                                                NDCGK_100         4220   
                                                                NDCGK_100         4349   
                                                                NDCGK_100         4223   

                                                                               score  
Algorithm                                          Timestamp    Metric                
AmazonMusic_TA_ItemKNN_Inc_Vaz_bdd640fb-0667-4a... t=1546241746 PrecisionK_100   0.0  
                                                                PrecisionK_100   0.0  
                                                                PrecisionK_100  0.01  
                                                                PrecisionK_100   0.0  
                                                                PrecisionK_100   0.0  
...                                                                              ...  
                                                   t=1672385746 NDCGK_100        0.0  
                                                                NDCGK_100        0.0  
                                                                NDCGK_100        0.0  
                                                                NDCGK_100        0.0  
                                                                NDCGK_100        0.0  

[2283 rows x 2 columns]

In [24]:
evaluator.metric_results("window")

window_score  \
Algorithm                                          Timestamp    Metric                         
AmazonMusic_TA_ItemKNN_Inc_Vaz_bdd640fb-0667-4a... t=1546241746 PrecisionK_100      0.000896   
                                                                RecallK_100         0.047678   
                                                                NDCGK_100           0.021485   
                                                   t=1577777746 PrecisionK_100      0.001037   
                                                                RecallK_100         0.069614   
                                                                NDCGK_100           0.028078   
                                                   t=1609313746 PrecisionK_100      0.000843   
                                                                RecallK_100         0.058052   
                                                                NDCGK_100           0.028766   
                                                   t=1640849746 PrecisionK_100      0.001027   
                                                                RecallK_100         0.074486   
                                                                NDCGK_100           0.015830   
                                                   t=1672385746 PrecisionK_100      0.001389   
                                                                RecallK_100         0.076389   
                                                                NDCGK_100           0.038740   

                                                                                num_user  
Algorithm                                          Timestamp    Metric                    
AmazonMusic_TA_ItemKNN_Inc_Vaz_bdd640fb-0667-4a... t=1546241746 PrecisionK_100       201  
                                                                RecallK_100          201  
                                                                NDCGK_100            201  
                                                   t=1577777746 PrecisionK_100       164  
                                                                RecallK_100          164  
                                                                NDCGK_100            164  
                                                   t=1609313746 PrecisionK_100       178  
                                                                RecallK_100          178  
                                                                NDCGK_100            178  
                                                   t=1640849746 PrecisionK_100       146  
                                                                RecallK_100          146  
                                                                NDCGK_100            146  
                                                   t=1672385746 PrecisionK_100        72  
                                                                RecallK_100           72  
                                                                NDCGK_100             72